In [1]:
from datasets import load_dataset

dataset = load_dataset("moatazhamza194/gb_train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/178M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1782961 [00:00<?, ? examples/s]

In [2]:
data = dataset["train"].to_pandas()

In [3]:
data.head()

,comment,label
0,"This is so cool. It's like, 'would you want yo...",0.000000
1,Thank you!! This would make my life a lot less...,0.000000
2,This is such an urgent design problem; kudos t...,0.000000
3,Is this something I'll be able to install on m...,0.000000
4,haha you guys are a bunch of losers.,0.893617


In [4]:
len(data)

1782961

In [5]:
import pandas as pd
import re
import json

In [6]:
def comment_gender_swap(comment, gendered_word_pairs):
    # Sort keys by length (longest first) to avoid partial matches
    keys = sorted(gendered_word_pairs.keys(), key=len, reverse=True)

    pattern = re.compile(
        r'(?<!\w)(?:' + '|'.join(map(re.escape, keys)) + r')(?=(?:\'s|’s)?(?![\w’\']))',
        flags=re.IGNORECASE
    )

    def replace(m):
        word = m.group(0)

        # Strip possessive for dictionary lookup
        base_word = re.sub(r"(\'s|’s)$", "", word, flags=re.IGNORECASE)

        if base_word.lower() not in gendered_word_pairs:
            return word  # no replacement

        base = gendered_word_pairs[base_word.lower()]

        # Preserve casing
        if word.isupper():
            replacement = base.upper()
        elif word[0].isupper():
            replacement = base.capitalize()
        else:
            replacement = base.lower()

        # Reattach possessive if needed
        if word.lower().endswith(("'s", "’s")):
            replacement += "'s"

        return replacement

    return pattern.sub(replace, comment)

def augment_with_gender_swap(df,pairs):

    # 1) produce swapped texts for all rows
    swapped_texts = df["comment"].astype(str).apply(lambda t: comment_gender_swap(t, pairs))

    # 2) find rows where text actually changed
    changed = ~swapped_texts.eq(df["comment"].astype(str))
    if not changed.any():
        return df.copy()

    # 3) take only changed rows, set new text
    aug = df.loc[changed].copy()
    aug["comment"] = swapped_texts.loc[changed].values

    # 5) return augmented rows
    return aug



In [7]:
def _compile_gender_mask_regex_from_terms(terms):
    """
    Build a single case-insensitive regex that matches any term
    (plus simple plural/possessive tails like s/'s/’s),
    but avoids false positives inside contractions or larger words.
    """
    if not terms:
        return re.compile(r"(?!x)x", flags=re.IGNORECASE)  # never matches

    # longest-first to avoid partial overlaps (e.g., 'herself' before 'her')
    vocab = sorted({t.lower() for t in terms}, key=len, reverse=True)

    pattern = r'(?<!\w)(?:%s)(?=(?:\'s|’s|s|es)?(?![\w’\']))' % "|".join(map(re.escape, vocab))

    return re.compile(pattern, flags=re.IGNORECASE)

def comment_gender_mask(text, genderTerms, token="[GENDER]"):
    """
    String-level masking (like comment_gender_swap but replaces with a fixed token).
    """
    rx = _compile_gender_mask_regex_from_terms(genderTerms)
    return rx.sub(token, str(text))

def augment_with_gender_mask(df, genderTerms, text_col="comment", token="[GENDER]"):
    """
    Returns original df + masked duplicates for rows whose text changed.
    - Uses the provided `terms` list (derived from your gendered pairs).
    - Zeros subgroup cols (male/female) on augmented rows.
    - No new columns are added.
    """
    #assert text_col in df.columns, f"Missing column: {text_col}"
    rx = _compile_gender_mask_regex_from_terms(genderTerms)

    # 1) produce masked texts for all rows
    df[text_col] = df[text_col].astype(str).apply(lambda t: rx.sub(token, t))

    return df # return only the masked data


In [9]:
# import the dictionary
with open("gendered_word_pairs.json", "r", encoding="utf-8") as f:
    gendered_word_pairs = json.load(f)

# add reverse pairs IN PLACE
for k, v in list(gendered_word_pairs.items()):
    gendered_word_pairs[v] = k

# build terms from your existing dict (both keys and values)
genderTerms = sorted({str(k).lower() for k in gendered_word_pairs.keys()} |
               {str(v).lower() for v in gendered_word_pairs.values()})

gender_swap_data = augment_with_gender_swap(data.copy(), gendered_word_pairs)
gender_mask_data = augment_with_gender_mask(data.copy(), gendered_word_pairs)

#print(data.head())
#print(gender_swap_data.head())
#print(gender_mask_data.head())

In [10]:
data.loc[248593]["comment"]

'And so another generation learns the hypocritical ways of the left. They should have just asked.'

In [11]:
gender_swap_data.loc[248593]["comment"]

KeyError: 248593

In [12]:
gender_mask_data.loc[248593]["comment"]

'And so another generation learns the hypocritical ways of the left. They should have just asked.'

In [13]:
# add the gender swapped rows to the original data
gender_swap_data = pd.concat([data, gender_swap_data], ignore_index=True)

In [14]:
gender_swap_data.head()

,comment,label
0,"This is so cool. It's like, 'would you want yo...",0.000000
1,Thank you!! This would make my life a lot less...,0.000000
2,This is such an urgent design problem; kudos t...,0.000000
3,Is this something I'll be able to install on m...,0.000000
4,haha you guys are a bunch of losers.,0.893617


In [15]:
from datasets import Dataset
gender_swap_dataset = Dataset.from_pandas(gender_swap_data)
gender_mask_dataset = Dataset.from_pandas(gender_mask_data)

In [16]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `new` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as n

In [17]:
gender_swap_dataset.push_to_hub("moatazhamza194/gb_train_swap")
gender_mask_dataset.push_to_hub("moatazhamza194/gb_train_mask")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1200 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   3%|3         | 8.29MB /  247MB            

...

Creating parquet from Arrow format:   0%|          | 0/892 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   2%|2         | 3.67MB /  178MB            

CommitInfo(commit_url='https://huggingface.co/datasets/moatazhamza194/gb_train_mask/commit/4c6ca4b82aa3f6256535e02f785e86e9cd372b97', commit_message='Upload dataset', commit_description='', oid='4c6ca4b82aa3f6256535e02f785e86e9cd372b97', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/moatazhamza194/gb_train_mask', endpoint='https://huggingface.co', repo_type='dataset', repo_id='moatazhamza194/gb_train_mask'), pr_revision=None, pr_num=None)

In [18]:
len(gender_swap_data)

2398586

In [19]:
len(gender_mask_data)

1782961